<a href="https://colab.research.google.com/github/wheemin-2/25-1-ESAA/blob/main/SS_0523_CNN_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. CIFAR-10 데이터셋을 이용한 CNN 모델

CIFAR-10 (Canadian Institute for Advanced Research 10) 데이터셋은 머신러닝 및 컴퓨터 비전 연구에서 널리 사용되는 이미지 분류 데이터셋입니다. 이 데이터셋은 10개의 상이한 카테고리로 분류된 컬러 이미지들을 포함하고 있으며, 주로 딥러닝 모델의 이미지 분류 성능을 평가하는 데 사용됩니다.

*CIFAR-10의 주요 특징*
- 이미지 수: 총 60,000장
- 훈련 이미지: 50,000장
- 테스트 이미지: 10,000장
- 이미지 크기: 32x32 픽셀, RGB 컬러 (채널 수: 3)
- 클래스 수: 10개
- 클래스 목록: airplane (비행기), automobile (자동차), bird (새), cat (고양이), deer (사슴), dog (개), frog (개구리), horse (말), ship (배), truck (트럭)

*데이터 구성*
- 각 클래스마다 이미지가 균등하게 분포되어 있어, 클래스당 6,000장의 이미지가 존재합니다.
- 훈련 세트는 클래스당 5,000장, 테스트 세트는 클래스당 1,000장으로 구성되어 있습니다.

## Step 1. 필요한 라이브러리 임포트

In [1]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [4]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
# 디바이스 확인
device

device(type='cpu')

## Step 2. CIFAR-10 데이터셋 다운로드

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화 추가
])

# 데이터셋 로드
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)


100%|██████████| 170M/170M [00:03<00:00, 54.8MB/s]


##Step 3. DataLoader 설정

In [7]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

## Step 4. CNN 모델 정의

In [9]:
# CNN 모델 정의

class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()

        # 첫 번째 합성곱 층
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1) #‼️32로 시작
        self.bn1 = nn.BatchNorm2d(32)  # 배치 정규화 채널 수 입력‼️

        # 두 번째 합성곱 층
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(64)

        # 세 번째 합성곱 층
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128 , kernel_size=3)
        self.bn3 = nn.BatchNorm2d(128)

        # 풀링 층 (MaxPooling)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer
        self.fc1 = nn.Linear(128*2*2, 256)  # CIFAR-10은 32x32 크기
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # 활성화 함수 적용
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)  # Flatten

        x = F.relu(self.fc1(x))  # Fully Connected 활성화 함수 추가
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## Step 5. 모델 학습 설정

In [11]:
# 모델을 초기화하세요.
learning_rate = 0.001
model = CIFAR10_CNN()
model.to(device)

# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

CIFAR10_CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


## Step 6. 모델 학습 루프

In [13]:
# 모델 학습을 위한 루프를 작성하세요. 에폭 수는 3로 설정합니다.
# 런타임 약 5분
num_epochs = 3
for epoch in range(num_epochs):
   for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 모델을 통해 예측값을 계산
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 경사 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 최적화
        optimizer.step()

   print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/3], Loss: 1.5178
Epoch [2/3], Loss: 1.1124
Epoch [3/3], Loss: 0.6994


## Step 7. 테스트 정확도 평가

In [14]:
# 모델의 성능 평가

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += len(labels)
        correct += (predicted == labels).sum()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 71.81%


# 2. MNIST 데이터셋 분류 모델




## Step 1. 필요한 라이브러리 임포트

In [15]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [16]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Step 2. MNIST 데이터셋 다운로드

In [17]:
# MNIST 데이터셋을 다운로드하고, 텐서로 변환할 수 있도록 필요한 전처리 함수를 추가하세요.
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 20.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 622kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 5.61MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.85MB/s]


##Step 3. DataLoader 설정

In [18]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

## Step 4. CNN 모델 정의

In [21]:
# CNN 모델 정의
class MNISTCNN(nn.Module):
    def __init__(self):
        super(MNISTCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

## Step 5. 모델 학습 설정

In [24]:
# 모델을 초기화하세요.
model2 = MNISTCNN()

# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model2.parameters(), lr=0.001)


## Step 6. 모델 학습 루프

In [25]:
# 모델 학습을 위한 루프를 작성하세요. 에폭 수는 3로 설정합니다.
# 런타임 약 5분
num_epochs = 3
for epoch in range(num_epochs):
   for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 모델을 통해 예측값을 계산
        outputs = model2(images)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 경사 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 최적화
        optimizer.step()

   print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch [1/3], Loss: 0.2733
Epoch [2/3], Loss: 0.0042
Epoch [3/3], Loss: 0.0269


## Step 7. 테스트 정확도 평가

In [26]:
# 모델의 성능 평가
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model2(images)
        _, predicted = torch.max(outputs.data, 1)

        total += len(labels)
        correct += (predicted == labels).sum()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Test Accuracy: 98.59%
